In [1]:
import pandas as pd
import numpy as np
from utils.data_helper import get_markable_dataframe, get_embedding_variables
from model_builders.coreference_classifier import CoreferenceClassifierModelBuilder
from functools import reduce
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
embedding_indexes_file_path = 'helper_files/embedding/embedding_indexes.txt'
indexed_embedding_file_path = 'helper_files/embedding/indexed_embedding.txt'

word_vector, embedding_matrix, idx_by_word, word_by_idx = get_embedding_variables(embedding_indexes_file_path, indexed_embedding_file_path)

In [3]:
markables = get_markable_dataframe("data/training/markables.csv", word_vector, idx_by_word)

markables.head()

,id,text,is_pronoun,entity_type,is_proper_name,is_first_person,num_words,previous_words,next_words,is_singleton
0,1,[1],0,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]",0,0,1,[],"[2, 3, 4, 5, 6, 7, 8, 9, 10, 2]","[0.0, 1.0]"
1,2,"[4, 5]",0,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]",0,0,2,"[1, 2, 3]","[6, 7, 8, 9, 10, 2, 11, 10, 12, 13]","[0.0, 1.0]"
2,3,"[6, 7, 8]",0,"[0, 0, 0, 1, 0, 0, 0, 1, 0, 0]",1,0,3,"[1, 2, 3, 4, 5]","[9, 10, 2, 11, 10, 12, 13, 14, 15, 16]","[1.0, 0.0]"
3,4,[10],0,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]",0,0,1,"[1, 2, 3, 4, 5, 6, 7, 8, 9]","[2, 11, 10, 12, 13, 14, 15, 16, 17, 18]","[0.0, 1.0]"
4,5,"[10, 12]",0,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]",0,0,2,"[3, 4, 5, 6, 7, 8, 9, 10, 2, 11]","[13, 14, 15, 16, 17, 18, 4, 5, 19, 20]","[0.0, 1.0]"


In [4]:
# pairs_budi = pd.read_csv("data/training/mention_pairs_budi.csv")
pairs_soon = pd.read_csv("data/training/mention_pairs_soon.csv")
pairs_gilang = pd.read_csv("data/training/mention_pairs_gilang.csv")

pairs_soon.head()

,m1_id,m2_id,is_exact_match,is_words_match,is_substring,is_abbreviation,is_appositive,is_nearest_candidate,sentence_distance,word_distance,markable_distance,is_coreference
0,3,11,0,0,0,0,0,0,2,31,8,1
1,4,11,0,0,0,0,0,0,2,29,7,0
2,5,11,0,0,0,0,0,0,2,25,6,0
3,6,11,0,0,0,0,0,0,2,16,5,0
4,7,11,0,0,0,0,0,0,1,14,4,0


In [5]:
max_text_length = 10
max_prev_words_length = 10
max_next_words_length = 10

def get_data(markable_ids):
    indices = reduce(lambda a, b: a + [b], map(lambda a: markables.index[markables['id'] == a].tolist()[0], markable_ids), [])
    data = markables.loc[indices]
    
    data_text = pad_sequences(data.text, maxlen=max_text_length, padding='post')
    data_previous_words = pad_sequences(data.previous_words.map(lambda seq: seq[(-1*max_prev_words_length):]), maxlen=max_prev_words_length, padding='pre')
    data_next_words = pad_sequences(data.next_words.map(lambda seq: seq[:max_next_words_length]), maxlen=max_next_words_length, padding='post')
    data_syntactic = data[['is_pronoun', 'entity_type', 'is_proper_name', 'is_first_person']]

    data_syntactic = np.array(list(map(lambda p: reduce(lambda x,y: x + y, [i if type(i) is list else [i] for i in p]), data_syntactic.values)))
    is_singleton = np.vstack(data.is_singleton)
    
    return data_text, data_previous_words, data_next_words, data_syntactic, is_singleton

def get_pair_data(markable_ids_1, markable_ids_2):
    text_1, prev_1, next_1, syntactic_1, is_singleton_1 = get_data(markable_ids_1)
    text_2, prev_2, next_2, syntactic_2, is_singleton_2 = get_data(markable_ids_2)
    
    return text_1, text_2, prev_1, prev_2, next_1, next_2, syntactic_1, syntactic_2, is_singleton_1, is_singleton_2

def get_relation_data(mention_pairs):
    return mention_pairs[['is_exact_match', 'is_words_match', 'is_substring', 'is_abbreviation', 'is_appositive', 'is_nearest_candidate', 'sentence_distance', 'word_distance', 'markable_distance']]

# Soon-Generated

In [6]:
text_11, text_21, prev_11, prev_21, next_11, next_21, syntactic_11, syntactic_21, is_singleton_11, is_singleton_21 = get_pair_data(pairs_soon.m1_id, pairs_soon.m2_id)
relation_1 = get_relation_data(pairs_soon)
label_soon = np.vstack(to_categorical(pairs_soon.is_coreference, num_classes=2))

## Build Model

### Word

In [7]:
words_model_builder = CoreferenceClassifierModelBuilder(
    use_words_feature=True, 
    use_context_feature=False, 
    use_syntactic_feature=False,
    use_relation_feature=False,
    embedding_matrix=embedding_matrix
)

In [10]:
words_model_1 = words_model_builder.create_model()

In [11]:
words_model_1.fit([text_11, text_21], label_soon, epochs=5)
words_model_1.save('models/coreference_classifiers/words_soon_5.model')

Epoch 1/5
5548/5548==============================] - 2s 395us/sample - loss: 0.3300 - acc: 0.8733
Epoch 2/5
5548/5548==============================] - 2s 335us/sample - loss: 0.2342 - acc: 0.8877
Epoch 3/5
5548/5548==============================] - 2s 338us/sample - loss: 0.2097 - acc: 0.8868
Epoch 4/5
5548/5548==============================] - 2s 336us/sample - loss: 0.1851 - acc: 0.8998
Epoch 5/5
5548/5548==============================] - 2s 335us/sample - loss: 0.1690 - acc: 0.9232


In [12]:
words_model_1.fit([text_11, text_21], label_soon, epochs=5)
words_model_1.save('models/coreference_classifiers/words_soon_10.model')

Epoch 1/5
5548/5548==============================] - 2s 333us/sample - loss: 0.1658 - acc: 0.9254
Epoch 2/5
5548/5548==============================] - 2s 332us/sample - loss: 0.1599 - acc: 0.9299
Epoch 3/5
5548/5548==============================] - 2s 336us/sample - loss: 0.1442 - acc: 0.9340
Epoch 4/5
5548/5548==============================] - 2s 334us/sample - loss: 0.1494 - acc: 0.9337
Epoch 5/5
5548/5548==============================] - 2s 336us/sample - loss: 0.1350 - acc: 0.9403


In [13]:
words_model_1.fit([text_11, text_21], label_soon, epochs=10)
words_model_1.save('models/coreference_classifiers/words_soon_20.model')

Epoch 1/10
5548/5548==============================] - 2s 331us/sample - loss: 0.1336 - acc: 0.9389
Epoch 2/10
5548/5548==============================] - 2s 339us/sample - loss: 0.1376 - acc: 0.9389
Epoch 3/10
5548/5548==============================] - 2s 334us/sample - loss: 0.1294 - acc: 0.9456
Epoch 4/10
5548/5548==============================] - 2s 337us/sample - loss: 0.1222 - acc: 0.9414
Epoch 5/10
5548/5548==============================] - 2s 336us/sample - loss: 0.1152 - acc: 0.9463
Epoch 6/10
5548/5548==============================] - 2s 333us/sample - loss: 0.1155 - acc: 0.9459
Epoch 7/10
5548/5548==============================] - 2s 334us/sample - loss: 0.1126 - acc: 0.9481
Epoch 8/10
5548/5548==============================] - 2s 333us/sample - loss: 0.1157 - acc: 0.9463
Epoch 9/10
5548/5548==============================] - 2s 336us/sample - loss: 0.1102 - acc: 0.9461
Epoch 10/10
5548/5548==============================] - 2s 336us/sample - loss: 0.1111 - acc: 0.9528


### Context

In [14]:
context_model_builder = CoreferenceClassifierModelBuilder(
    use_words_feature=False, 
    use_context_feature=True, 
    use_syntactic_feature=False,
    use_relation_feature=False,
    embedding_matrix=embedding_matrix
)

In [15]:
context_model_1 = context_model_builder.create_model()

In [16]:
context_model_1.fit([prev_11, prev_21, next_11, next_21], label_soon, epochs=5)
context_model_1.save('models/coreference_classifiers/context_soon_5.model')

Epoch 1/5
5548/5548==============================] - 3s 608us/sample - loss: 0.3933 - acc: 0.8828
Epoch 2/5
5548/5548==============================] - 3s 516us/sample - loss: 0.3518 - acc: 0.8877
Epoch 3/5
5548/5548==============================] - 3s 502us/sample - loss: 0.3343 - acc: 0.8877
Epoch 4/5
5548/5548==============================] - 3s 510us/sample - loss: 0.2967 - acc: 0.8877
Epoch 5/5
5548/5548==============================] - 3s 511us/sample - loss: 0.2756 - acc: 0.8877


In [17]:
context_model_1.fit([prev_11, prev_21, next_11, next_21], label_soon, epochs=5)
context_model_1.save('models/coreference_classifiers/context_soon_10.model')

Epoch 1/5
5548/5548==============================] - 3s 501us/sample - loss: 0.2436 - acc: 0.8877
Epoch 2/5
5548/5548==============================] - 3s 514us/sample - loss: 0.2281 - acc: 0.8877
Epoch 3/5
5548/5548==============================] - 3s 510us/sample - loss: 0.2175 - acc: 0.8935
Epoch 4/5
5548/5548==============================] - 3s 510us/sample - loss: 0.2035 - acc: 0.9086
Epoch 5/5
5548/5548==============================] - 3s 508us/sample - loss: 0.1926 - acc: 0.9140


In [18]:
context_model_1.fit([prev_11, prev_21, next_11, next_21], label_soon, epochs=10)
context_model_1.save('models/coreference_classifiers/context_soon_20.model')

Epoch 1/10
5548/5548==============================] - 3s 511us/sample - loss: 0.1788 - acc: 0.9194
Epoch 2/10
5548/5548==============================] - 3s 513us/sample - loss: 0.1666 - acc: 0.9257
Epoch 3/10
5548/5548==============================] - 3s 513us/sample - loss: 0.1679 - acc: 0.9257
Epoch 4/10
5548/5548==============================] - 3s 509us/sample - loss: 0.1573 - acc: 0.9248
Epoch 5/10
5548/5548==============================] - 3s 511us/sample - loss: 0.1520 - acc: 0.9333
Epoch 6/10
5548/5548==============================] - 3s 501us/sample - loss: 0.1518 - acc: 0.9328
Epoch 7/10
5548/5548==============================] - 3s 508us/sample - loss: 0.1565 - acc: 0.9302
Epoch 8/10
5548/5548==============================] - 3s 510us/sample - loss: 0.1444 - acc: 0.9385
Epoch 9/10
5548/5548==============================] - 3s 516us/sample - loss: 0.1429 - acc: 0.9405
Epoch 10/10
5548/5548==============================] - 3s 506us/sample - loss: 0.1381 - acc: 0.9438


### Syntactic

In [19]:
syntactic_model_builder = CoreferenceClassifierModelBuilder(
    use_words_feature=False, 
    use_context_feature=False, 
    use_syntactic_feature=True,
    use_relation_feature=True,
    embedding_matrix=embedding_matrix,
    syntactic_features_num=syntactic_11.shape[1],
    relation_features_num=relation_1.shape[1]
)

In [20]:
syntactic_model_1 = syntactic_model_builder.create_model()

In [21]:
syntactic_model_1.fit([syntactic_11, syntactic_21, relation_1], label_soon, epochs=5)
syntactic_model_1.save('models/coreference_classifiers/syntactic_soon_5.model')

Epoch 1/5
5548/5548==============================] - 2s 322us/sample - loss: 0.6058 - acc: 0.8567
Epoch 2/5
5548/5548==============================] - 1s 259us/sample - loss: 0.2885 - acc: 0.8805
Epoch 3/5
5548/5548==============================] - 1s 262us/sample - loss: 0.2595 - acc: 0.8841
Epoch 4/5
5548/5548==============================] - 1s 266us/sample - loss: 0.2441 - acc: 0.8852
Epoch 5/5
5548/5548==============================] - 1s 258us/sample - loss: 0.2379 - acc: 0.8861


In [22]:
syntactic_model_1.fit([syntactic_11, syntactic_21, relation_1], label_soon, epochs=5)
syntactic_model_1.save('models/coreference_classifiers/syntactic_soon_10.model')

Epoch 1/5
5548/5548==============================] - 1s 252us/sample - loss: 0.2329 - acc: 0.8870
Epoch 2/5
5548/5548==============================] - 1s 262us/sample - loss: 0.2306 - acc: 0.8872
Epoch 3/5
5548/5548==============================] - 1s 260us/sample - loss: 0.2263 - acc: 0.8879
Epoch 4/5
5548/5548==============================] - 1s 261us/sample - loss: 0.2206 - acc: 0.8902
Epoch 5/5
5548/5548==============================] - 1s 261us/sample - loss: 0.2198 - acc: 0.8933


In [23]:
syntactic_model_1.fit([syntactic_11, syntactic_21, relation_1], label_soon, epochs=10)
syntactic_model_1.save('models/coreference_classifiers/syntactic_soon_20.model')

Epoch 1/10
5548/5548==============================] - 1s 256us/sample - loss: 0.2116 - acc: 0.8967
Epoch 2/10
5548/5548==============================] - 1s 265us/sample - loss: 0.2056 - acc: 0.9016
Epoch 3/10
5548/5548==============================] - 1s 263us/sample - loss: 0.1998 - acc: 0.9099
Epoch 4/10
5548/5548==============================] - 1s 258us/sample - loss: 0.1819 - acc: 0.9256
Epoch 5/10
5548/5548==============================] - 1s 258us/sample - loss: 0.1832 - acc: 0.9221
Epoch 6/10
5548/5548==============================] - 1s 267us/sample - loss: 0.1733 - acc: 0.9265
Epoch 7/10
5548/5548==============================] - 1s 266us/sample - loss: 0.1614 - acc: 0.9315
Epoch 8/10
5548/5548==============================] - 1s 263us/sample - loss: 0.1604 - acc: 0.9315
Epoch 9/10
5548/5548==============================] - 1s 259us/sample - loss: 0.1585 - acc: 0.9301
Epoch 10/10
5548/5548==============================] - 1s 261us/sample - loss: 0.1551 - acc: 0.9329


### Words + Context

In [24]:
words_context_model_builder = CoreferenceClassifierModelBuilder(
    use_words_feature=True, 
    use_context_feature=True, 
    use_syntactic_feature=False,
    use_relation_feature=False,
    embedding_matrix=embedding_matrix,
)

In [25]:
words_context_model_1 = words_context_model_builder.create_model()

In [26]:
words_context_model_1.fit([text_11, text_21, prev_11, prev_21, next_11, next_21], label_soon, epochs=5)
words_context_model_1.save('models/coreference_classifiers/words_context_soon_5.model')

Epoch 1/5
5548/5548==============================] - 5s 856us/sample - loss: 0.3369 - acc: 0.8821
Epoch 2/5
5548/5548==============================] - 4s 713us/sample - loss: 0.2273 - acc: 0.8895
Epoch 3/5
5548/5548==============================] - 4s 719us/sample - loss: 0.1876 - acc: 0.9129
Epoch 4/5
5548/5548==============================] - 4s 723us/sample - loss: 0.1659 - acc: 0.9250
Epoch 5/5
5548/5548==============================] - 4s 716us/sample - loss: 0.1399 - acc: 0.9409


In [27]:
words_context_model_1.fit([text_11, text_21, prev_11, prev_21, next_11, next_21], label_soon, epochs=5)
words_context_model_1.save('models/coreference_classifiers/words_context_soon_10.model')

Epoch 1/5
5548/5548==============================] - 4s 703us/sample - loss: 0.1227 - acc: 0.9501
Epoch 2/5
5548/5548==============================] - 4s 718us/sample - loss: 0.0970 - acc: 0.9575
Epoch 3/5
5548/5548==============================] - 4s 720us/sample - loss: 0.1029 - acc: 0.9600
Epoch 4/5
5548/5548==============================] - 4s 722us/sample - loss: 0.0851 - acc: 0.9710
Epoch 5/5
5548/5548==============================] - 4s 728us/sample - loss: 0.0798 - acc: 0.9690


In [28]:
words_context_model_1.fit([text_11, text_21, prev_11, prev_21, next_11, next_21], label_soon, epochs=10)
words_context_model_1.save('models/coreference_classifiers/words_context_soon_20.model')

Epoch 1/10
5548/5548==============================] - 4s 708us/sample - loss: 0.0697 - acc: 0.9726
Epoch 2/10
5548/5548==============================] - 4s 718us/sample - loss: 0.0636 - acc: 0.9746
Epoch 3/10
5548/5548==============================] - 4s 716us/sample - loss: 0.0671 - acc: 0.9764
Epoch 4/10
5548/5548==============================] - 4s 713us/sample - loss: 0.0505 - acc: 0.9840
Epoch 5/10
5548/5548==============================] - 4s 718us/sample - loss: 0.0413 - acc: 0.9858
Epoch 6/10
5548/5548==============================] - 4s 721us/sample - loss: 0.0485 - acc: 0.9841
Epoch 7/10
5548/5548==============================] - 4s 714us/sample - loss: 0.0459 - acc: 0.9858
Epoch 8/10
5548/5548==============================] - 4s 718us/sample - loss: 0.0438 - acc: 0.9865
Epoch 9/10
5548/5548==============================] - 4s 723us/sample - loss: 0.0320 - acc: 0.9870
Epoch 10/10
5548/5548==============================] - 4s 721us/sample - loss: 0.0217 - acc: 0.9908


### Words + Syntactic

In [29]:
words_syntactic_model_builder = CoreferenceClassifierModelBuilder(
    use_words_feature=True, 
    use_context_feature=False, 
    use_syntactic_feature=True,
    use_relation_feature=True,
    embedding_matrix=embedding_matrix,
    syntactic_features_num=syntactic_11.shape[1],
    relation_features_num=relation_1.shape[1]
)

In [30]:
words_syntactic_model_1 = words_syntactic_model_builder.create_model()

In [31]:
words_syntactic_model_1.fit([text_11, text_21, syntactic_11, syntactic_21, relation_1], label_soon, epochs=5)
words_syntactic_model_1.save('models/coreference_classifiers/words_syntactic_soon_5.model')

Epoch 1/5
5548/5548==============================] - 3s 572us/sample - loss: 0.3510 - acc: 0.8791
Epoch 2/5
5548/5548==============================] - 3s 462us/sample - loss: 0.2248 - acc: 0.8928
Epoch 3/5
5548/5548==============================] - 3s 472us/sample - loss: 0.1945 - acc: 0.9081
Epoch 4/5
5548/5548==============================] - 3s 462us/sample - loss: 0.1815 - acc: 0.9155
Epoch 5/5
5548/5548==============================] - 3s 465us/sample - loss: 0.1609 - acc: 0.9311


In [32]:
words_syntactic_model_1.fit([text_11, text_21, syntactic_11, syntactic_21, relation_1], label_soon, epochs=5)
words_syntactic_model_1.save('models/coreference_classifiers/words_syntactic_soon_10.model')

Epoch 1/5
5548/5548==============================] - 3s 459us/sample - loss: 0.1399 - acc: 0.9380
Epoch 2/5
5548/5548==============================] - 3s 465us/sample - loss: 0.1240 - acc: 0.9486
Epoch 3/5
5548/5548==============================] - 3s 465us/sample - loss: 0.1161 - acc: 0.9497
Epoch 4/5
5548/5548==============================] - 3s 462us/sample - loss: 0.1011 - acc: 0.9611
Epoch 5/5
5548/5548==============================] - 3s 466us/sample - loss: 0.0936 - acc: 0.9612


In [33]:
words_syntactic_model_1.fit([text_11, text_21, syntactic_11, syntactic_21, relation_1], label_soon, epochs=10)
words_syntactic_model_1.save('models/coreference_classifiers/words_syntactic_soon_20.model')

Epoch 1/10
5548/5548==============================] - 2s 449us/sample - loss: 0.0784 - acc: 0.9688
Epoch 2/10
5548/5548==============================] - 3s 464us/sample - loss: 0.0799 - acc: 0.9668
Epoch 3/10
5548/5548==============================] - 3s 462us/sample - loss: 0.0732 - acc: 0.9694
Epoch 4/10
5548/5548==============================] - 3s 462us/sample - loss: 0.0771 - acc: 0.9703
Epoch 5/10
5548/5548==============================] - 3s 469us/sample - loss: 0.0722 - acc: 0.9715
Epoch 6/10
5548/5548==============================] - 3s 465us/sample - loss: 0.0672 - acc: 0.9713
Epoch 7/10
5548/5548==============================] - 3s 464us/sample - loss: 0.0615 - acc: 0.9749
Epoch 8/10
5548/5548==============================] - 3s 465us/sample - loss: 0.0639 - acc: 0.9744
Epoch 9/10
5548/5548==============================] - 3s 468us/sample - loss: 0.0606 - acc: 0.9749
Epoch 10/10
5548/5548==============================] - 3s 463us/sample - loss: 0.0564 - acc: 0.9778


### Context + Syntactic

In [44]:
context_syntactic_model_builder = CoreferenceClassifierModelBuilder(
    use_words_feature=False, 
    use_context_feature=True, 
    use_syntactic_feature=True,
    use_relation_feature=True,
    embedding_matrix=embedding_matrix,
    syntactic_features_num=syntactic_11.shape[1],
    relation_features_num=relation_1.shape[1]
)

In [45]:
context_syntactic_model_1 = context_syntactic_model_builder.create_model()

In [46]:
context_syntactic_model_1.fit([prev_11, prev_21, next_11, next_21, syntactic_11, syntactic_21, relation_1], label_soon, epochs=5)
context_syntactic_model_1.save('models/coreference_classifiers/context_syntactic_soon_5.model')

Epoch 1/5
5548/5548==============================] - 4s 810us/sample - loss: 0.4754 - acc: 0.8410
Epoch 2/5
5548/5548==============================] - 4s 638us/sample - loss: 0.2656 - acc: 0.8845
Epoch 3/5
5548/5548==============================] - 4s 639us/sample - loss: 0.2417 - acc: 0.8854
Epoch 4/5
5548/5548==============================] - 4s 641us/sample - loss: 0.2225 - acc: 0.8906
Epoch 5/5
5548/5548==============================] - 4s 637us/sample - loss: 0.1999 - acc: 0.8989


In [47]:
context_syntactic_model_1.fit([prev_11, prev_21, next_11, next_21, syntactic_11, syntactic_21, relation_1], label_soon, epochs=5)
context_syntactic_model_1.save('models/coreference_classifiers/context_syntactic_soon_10.model')

Epoch 1/5
5548/5548==============================] - 4s 631us/sample - loss: 0.1781 - acc: 0.9227
Epoch 2/5
5548/5548==============================] - 4s 634us/sample - loss: 0.1530 - acc: 0.9337
Epoch 3/5
5548/5548==============================] - 3s 629us/sample - loss: 0.1365 - acc: 0.9456
Epoch 4/5
5548/5548==============================] - 4s 638us/sample - loss: 0.1321 - acc: 0.9468
Epoch 5/5
5548/5548==============================] - 3s 630us/sample - loss: 0.1090 - acc: 0.9540


In [48]:
context_syntactic_model_1.fit([prev_11, prev_21, next_11, next_21, syntactic_11, syntactic_21, relation_1], label_soon, epochs=10)
context_syntactic_model_1.save('models/coreference_classifiers/context_syntactic_soon_20.model')

Epoch 1/10
5548/5548==============================] - 3s 618us/sample - loss: 0.0986 - acc: 0.9638
Epoch 2/10
5548/5548==============================] - 4s 645us/sample - loss: 0.0831 - acc: 0.9658
Epoch 3/10
5548/5548==============================] - 3s 630us/sample - loss: 0.0797 - acc: 0.9712
Epoch 4/10
5548/5548==============================] - 3s 630us/sample - loss: 0.0756 - acc: 0.9717
Epoch 5/10
5548/5548==============================] - 4s 634us/sample - loss: 0.0646 - acc: 0.9769
Epoch 6/10
5548/5548==============================] - 4s 633us/sample - loss: 0.0568 - acc: 0.9791
Epoch 7/10
5548/5548==============================] - 4s 631us/sample - loss: 0.0511 - acc: 0.9831
Epoch 8/10
5548/5548==============================] - 4s 632us/sample - loss: 0.0573 - acc: 0.9809
Epoch 9/10
5548/5548==============================] - 4s 634us/sample - loss: 0.0496 - acc: 0.9800
Epoch 10/10
5548/5548==============================] - 4s 633us/sample - loss: 0.0453 - acc: 0.9854


### Words + Context + Syntactic

In [39]:
words_context_syntactic_model_builder = CoreferenceClassifierModelBuilder(
    use_words_feature=True, 
    use_context_feature=True, 
    use_syntactic_feature=True,
    use_relation_feature=True,
    embedding_matrix=embedding_matrix,
    syntactic_features_num=syntactic_11.shape[1],
    relation_features_num=relation_1.shape[1]
)

In [40]:
words_context_syntactic_model_1 = words_context_syntactic_model_builder.create_model()

In [41]:
words_context_syntactic_model_1.fit([text_11, text_21, prev_11, prev_21, next_11, next_21, syntactic_11, syntactic_21, relation_1], label_soon, epochs=5)
words_context_syntactic_model_1.save('models/coreference_classifiers/words_context_syntactic_soon_5.model')

Epoch 1/5
5548/5548==============================] - 6s 1ms/sample - loss: 0.3265 - acc: 0.8852
Epoch 2/5
5548/5548==============================] - 5s 841us/sample - loss: 0.2202 - acc: 0.8890
Epoch 3/5
5548/5548==============================] - 5s 848us/sample - loss: 0.1809 - acc: 0.9090
Epoch 4/5
5548/5548==============================] - 5s 836us/sample - loss: 0.1593 - acc: 0.9281
Epoch 5/5
5548/5548==============================] - 5s 846us/sample - loss: 0.1351 - acc: 0.9432


In [42]:
words_context_syntactic_model_1.fit([text_11, text_21, prev_11, prev_21, next_11, next_21, syntactic_11, syntactic_21, relation_1], label_soon, epochs=5)
words_context_syntactic_model_1.save('models/coreference_classifiers/words_context_syntactic_soon_10.model')

Epoch 1/5
5548/5548==============================] - 5s 819us/sample - loss: 0.1162 - acc: 0.9542
Epoch 2/5
5548/5548==============================] - 5s 849us/sample - loss: 0.1026 - acc: 0.9620
Epoch 3/5
5548/5548==============================] - 5s 843us/sample - loss: 0.0904 - acc: 0.9674
Epoch 4/5
5548/5548==============================] - 5s 839us/sample - loss: 0.0765 - acc: 0.9701
Epoch 5/5
5548/5548==============================] - 5s 851us/sample - loss: 0.0626 - acc: 0.9789


In [43]:
words_context_syntactic_model_1.fit([text_11, text_21, prev_11, prev_21, next_11, next_21, syntactic_11, syntactic_21, relation_1], label_soon, epochs=10)
words_context_syntactic_model_1.save('models/coreference_classifiers/words_context_syntactic_soon_20.model')

Epoch 1/10
5548/5548==============================] - 5s 820us/sample - loss: 0.0544 - acc: 0.9798
Epoch 2/10
5548/5548==============================] - 5s 847us/sample - loss: 0.0509 - acc: 0.9840
Epoch 3/10
5548/5548==============================] - 5s 844us/sample - loss: 0.0467 - acc: 0.9867
Epoch 4/10
5548/5548==============================] - 5s 849us/sample - loss: 0.0479 - acc: 0.9854
Epoch 5/10
5548/5548==============================] - 5s 851us/sample - loss: 0.0369 - acc: 0.9859
Epoch 6/10
5548/5548==============================] - 5s 843us/sample - loss: 0.0360 - acc: 0.9901
Epoch 7/10
5548/5548==============================] - 5s 845us/sample - loss: 0.0327 - acc: 0.9894
Epoch 8/10
5548/5548==============================] - 5s 838us/sample - loss: 0.0284 - acc: 0.9903
Epoch 9/10
5548/5548==============================] - 5s 841us/sample - loss: 0.0282 - acc: 0.9901
Epoch 10/10
5548/5548==============================] - 5s 842us/sample - loss: 0.0327 - acc: 0.9885


# Gilang-Generated

In [49]:
text_12, text_22, prev_12, prev_22, next_12, next_22, syntactic_12, syntactic_22, is_singleton_12, is_singleton_22 = get_pair_data(pairs_gilang.m1_id, pairs_gilang.m2_id)
relation_2 = get_relation_data(pairs_gilang)
label_gilang = np.vstack(to_categorical(pairs_gilang.is_coreference, num_classes=2))

## Build Model

### Words

In [50]:
words_model_2 = words_model_builder.create_model()

In [51]:
words_model_2.fit([text_12, text_22], label_gilang, epochs=5)
words_model_2.save('models/coreference_classifiers/words_gilang_5.model')

Epoch 1/5
43625/43625==============================] - 17s 390us/sample - loss: 0.0670 - acc: 0.9855
Epoch 2/5
43625/43625==============================] - 16s 375us/sample - loss: 0.0473 - acc: 0.9857
Epoch 3/5
43625/43625==============================] - 16s 374us/sample - loss: 0.0453 - acc: 0.9857
Epoch 4/5
43625/43625==============================] - 16s 374us/sample - loss: 0.0414 - acc: 0.9857
Epoch 5/5
43625/43625==============================] - 16s 375us/sample - loss: 0.0421 - acc: 0.9857


In [52]:
words_model_2.fit([text_12, text_22], label_gilang, epochs=5)
words_model_2.save('models/coreference_classifiers/words_gilang_10.model')

Epoch 1/5
43625/43625==============================] - 16s 373us/sample - loss: 0.0395 - acc: 0.9857
Epoch 2/5
43625/43625==============================] - 16s 373us/sample - loss: 0.0396 - acc: 0.9857
Epoch 3/5
43625/43625==============================] - 16s 374us/sample - loss: 0.0378 - acc: 0.9857
Epoch 4/5
43625/43625==============================] - 16s 374us/sample - loss: 0.0368 - acc: 0.9857
Epoch 5/5
43625/43625==============================] - 16s 372us/sample - loss: 0.0375 - acc: 0.9857


In [53]:
words_model_2.fit([text_12, text_22], label_gilang, epochs=10)
words_model_2.save('models/coreference_classifiers/words_gilang_20.model')

Epoch 1/10
43625/43625==============================] - 16s 374us/sample - loss: 0.0363 - acc: 0.9857
Epoch 2/10
43625/43625==============================] - 16s 373us/sample - loss: 0.0363 - acc: 0.9857
Epoch 3/10
43625/43625==============================] - 16s 375us/sample - loss: 0.0349 - acc: 0.9857
Epoch 4/10
43625/43625==============================] - 16s 373us/sample - loss: 0.0343 - acc: 0.9857
Epoch 5/10
43625/43625==============================] - 16s 370us/sample - loss: 0.0339 - acc: 0.9857
Epoch 6/10
43625/43625==============================] - 16s 372us/sample - loss: 0.0348 - acc: 0.9857
Epoch 7/10
43625/43625==============================] - 16s 375us/sample - loss: 0.0346 - acc: 0.9857
Epoch 8/10
43625/43625==============================] - 16s 373us/sample - loss: 0.0336 - acc: 0.9857
Epoch 9/10
43625/43625==============================] - 16s 374us/sample - loss: 0.0374 - acc: 0.9857
Epoch 10/10
43625/43625==============================] - 16s 374us/sample - loss: 

### Context

In [54]:
context_model_2 = context_model_builder.create_model()

In [ ]:
context_model_2.fit([prev_12, prev_22, next_12, next_22], label_gilang, epochs=5)
context_model_2.save('models/coreference_classifiers/context_gilang_5.model')

Epoch 1/5
43625/43625==============================] - 23s 535us/sample - loss: 0.0858 - acc: 0.9857
Epoch 2/5
43625/43625==============================] - 24s 553us/sample - loss: 0.0721 - acc: 0.9857
Epoch 3/5
43625/43625==============================] - 24s 556us/sample - loss: 0.0660 - acc: 0.9857
Epoch 4/5
43625/43625==============================] - 24s 555us/sample - loss: 0.0605 - acc: 0.9857
Epoch 5/5
43625/43625==============================] - 24s 539us/sample - loss: 0.0558 - acc: 0.9857


In [ ]:
context_model_2.fit([prev_12, prev_22, next_12, next_22], label_gilang, epochs=5)
context_model_2.save('models/coreference_classifiers/context_gilang_10.model')

Epoch 1/5
43625/43625==============================] - 23s 527us/sample - loss: 0.0540 - acc: 0.9857
Epoch 2/5
43625/43625==============================] - 24s 556us/sample - loss: 0.0502 - acc: 0.9857
Epoch 3/5
43625/43625==============================] - 24s 555us/sample - loss: 0.0481 - acc: 0.9857
Epoch 4/5
43625/43625==============================] - 24s 549us/sample - loss: 0.0470 - acc: 0.9857
Epoch 5/5
43625/43625==============================] - 24s 552us/sample - loss: 0.0454 - acc: 0.9857


In [ ]:
context_model_2.fit([prev_12, prev_22, next_12, next_22], label_gilang, epochs=10)
context_model_2.save('models/coreference_classifiers/context_gilang_20.model')

Epoch 1/10
43625/43625==============================] - 24s 551us/sample - loss: 0.0436 - acc: 0.9857
Epoch 2/10
43625/43625==============================] - 24s 554us/sample - loss: 0.0436 - acc: 0.9857
Epoch 3/10
43625/43625==============================] - 24s 553us/sample - loss: 0.0431 - acc: 0.9857
Epoch 4/10
43625/43625==============================] - 24s 553us/sample - loss: 0.0418 - acc: 0.9857
Epoch 5/10
43625/43625==============================] - 24s 556us/sample - loss: 0.0409 - acc: 0.9857
Epoch 6/10
43625/43625==============================] - 24s 554us/sample - loss: 0.0405 - acc: 0.9857
Epoch 7/10
43625/43625==============================] - 24s 554us/sample - loss: 0.0391 - acc: 0.9857
Epoch 8/10
43625/43625==============================] - 24s 552us/sample - loss: 0.0394 - acc: 0.9857
Epoch 9/10
43625/43625==============================] - 24s 554us/sample - loss: 0.0398 - acc: 0.9857
Epoch 10/10
43625/43625==============================] - 24s 550us/sample - loss: 

### Syntactic

In [ ]:
syntactic_model_2 = syntactic_model_builder.create_model()

In [ ]:
syntactic_model_2.fit([syntactic_12, syntactic_22, relation_2], label_gilang, epochs=5)
syntactic_model_2.save('models/coreference_classifiers/syntactic_gilang_5.model')

Epoch 1/5
43625/43625==============================] - 14s 315us/sample - loss: 0.0659 - acc: 0.9857
Epoch 2/5
43625/43625==============================] - 13s 300us/sample - loss: 0.0505 - acc: 0.9857
Epoch 3/5
43625/43625==============================] - 13s 299us/sample - loss: 0.0476 - acc: 0.9857
Epoch 4/5
43625/43625==============================] - 13s 299us/sample - loss: 0.0442 - acc: 0.9857
Epoch 5/5
43625/43625==============================] - 13s 298us/sample - loss: 0.0431 - acc: 0.9857


In [ ]:
syntactic_model_2.fit([syntactic_12, syntactic_22, relation_2], label_gilang, epochs=5)
syntactic_model_2.save('models/coreference_classifiers/syntactic_gilang_10.model')

Epoch 1/5
43625/43625==============================] - 13s 297us/sample - loss: 0.0413 - acc: 0.9857
Epoch 2/5
43625/43625==============================] - 13s 298us/sample - loss: 0.0406 - acc: 0.9857
Epoch 3/5
43625/43625==============================] - 13s 298us/sample - loss: 0.0404 - acc: 0.9857
Epoch 4/5
43625/43625==============================] - 13s 299us/sample - loss: 0.0410 - acc: 0.9857
Epoch 5/5
43625/43625==============================] - 13s 300us/sample - loss: 0.0402 - acc: 0.9857


In [ ]:
syntactic_model_2.fit([syntactic_12, syntactic_22, relation_2], label_gilang, epochs=10)
syntactic_model_2.save('models/coreference_classifiers/syntactic_gilang_20.model')

Epoch 1/10
43625/43625==============================] - 13s 297us/sample - loss: 0.0395 - acc: 0.9857
Epoch 2/10
43625/43625==============================] - 13s 299us/sample - loss: 0.0397 - acc: 0.9857
Epoch 3/10
43625/43625==============================] - 13s 299us/sample - loss: 0.0389 - acc: 0.9857
Epoch 4/10
43625/43625==============================] - 13s 300us/sample - loss: 0.0390 - acc: 0.9857
Epoch 5/10
43625/43625==============================] - 13s 299us/sample - loss: 0.0388 - acc: 0.9857
Epoch 6/10
43625/43625==============================] - 13s 300us/sample - loss: 0.0394 - acc: 0.9857
Epoch 7/10
43625/43625==============================] - 13s 297us/sample - loss: 0.0391 - acc: 0.9857
Epoch 8/10
43625/43625==============================] - 13s 301us/sample - loss: 0.0389 - acc: 0.9857
Epoch 9/10
43625/43625==============================] - 13s 300us/sample - loss: 0.0391 - acc: 0.9857
Epoch 10/10
43625/43625==============================] - 13s 298us/sample - loss: 

### Words + Context

In [ ]:
words_context_model_2 = words_context_model_builder.create_model()

In [ ]:
words_context_model_2.fit([text_12, text_22, prev_12, prev_22, next_12, next_22], label_gilang, epochs=5)
words_context_model_2.save('models/coreference_classifiers/words_context_gilang_5.model')

Epoch 1/5
43625/43625==============================] - 32s 737us/sample - loss: 0.0673 - acc: 0.9857
Epoch 2/5
43625/43625==============================] - 31s 713us/sample - loss: 0.0481 - acc: 0.9857
Epoch 3/5
43625/43625==============================] - 31s 711us/sample - loss: 0.0455 - acc: 0.9857
Epoch 4/5
43625/43625==============================] - 31s 711us/sample - loss: 0.0424 - acc: 0.9857
Epoch 5/5
43625/43625==============================] - 31s 711us/sample - loss: 0.0400 - acc: 0.9857


In [ ]:
words_context_model_2.fit([text_12, text_22, prev_12, prev_22, next_12, next_22], label_gilang, epochs=5)
words_context_model_2.save('models/coreference_classifiers/words_context_gilang_10.model')

Epoch 1/5
43625/43625==============================] - 31s 711us/sample - loss: 0.0396 - acc: 0.9857
Epoch 2/5
43625/43625==============================] - 31s 710us/sample - loss: 0.0375 - acc: 0.9857
Epoch 3/5
43625/43625==============================] - 31s 713us/sample - loss: 0.0356 - acc: 0.9857
Epoch 4/5
43625/43625==============================] - 31s 712us/sample - loss: 0.0348 - acc: 0.9857
Epoch 5/5
43625/43625==============================] - 31s 712us/sample - loss: 0.0340 - acc: 0.9857


In [ ]:
words_context_model_2.fit([text_12, text_22, prev_12, prev_22, next_12, next_22], label_gilang, epochs=10)
words_context_model_2.save('models/coreference_classifiers/words_context_gilang_20.model')

Epoch 1/10
43625/43625==============================] - 31s 711us/sample - loss: 0.0330 - acc: 0.9857
Epoch 2/10
43625/43625==============================] - 31s 711us/sample - loss: 0.0342 - acc: 0.9857
Epoch 3/10
43625/43625==============================] - 31s 713us/sample - loss: 0.0333 - acc: 0.9857
Epoch 4/10
43625/43625==============================] - 31s 712us/sample - loss: 0.0300 - acc: 0.9857
Epoch 5/10
43625/43625==============================] - 31s 710us/sample - loss: 0.0311 - acc: 0.9857
Epoch 6/10
43625/43625==============================] - 31s 713us/sample - loss: 0.0300 - acc: 0.9857
Epoch 7/10
43625/43625==============================] - 31s 708us/sample - loss: 0.0304 - acc: 0.9857
Epoch 8/10
43625/43625==============================] - 31s 710us/sample - loss: 0.0301 - acc: 0.9858
Epoch 9/10
43625/43625==============================] - 31s 712us/sample - loss: 0.0300 - acc: 0.9857
Epoch 10/10
43625/43625==============================] - 31s 711us/sample - loss: 

### Words + Syntactic

In [ ]:
words_syntactic_model_2 = words_syntactic_model_builder.create_model()

In [ ]:
words_syntactic_model_2.fit([text_12, text_22, syntactic_12, syntactic_22, relation_2], label_gilang, epochs=5)
words_syntactic_model_2.save('models/coreference_classifiers/words_syntactic_gilang_5.model')

Epoch 1/5
43625/43625==============================] - 23s 524us/sample - loss: 0.0635 - acc: 0.9856
Epoch 2/5
43625/43625==============================] - 22s 498us/sample - loss: 0.0464 - acc: 0.9857
Epoch 3/5
43625/43625==============================] - 22s 502us/sample - loss: 0.0435 - acc: 0.9857
Epoch 4/5
43625/43625==============================] - 22s 501us/sample - loss: 0.0405 - acc: 0.9857
Epoch 5/5
43625/43625==============================] - 22s 501us/sample - loss: 0.0389 - acc: 0.9857


In [ ]:
words_syntactic_model_2.fit([text_12, text_22, syntactic_12, syntactic_22, relation_2], label_gilang, epochs=5)
words_syntactic_model_2.save('models/coreference_classifiers/words_syntactic_gilang_10.model')

Epoch 1/5
43625/43625==============================] - 22s 501us/sample - loss: 0.0384 - acc: 0.9857
Epoch 2/5
43625/43625==============================] - 22s 503us/sample - loss: 0.0368 - acc: 0.9857
Epoch 3/5
43625/43625==============================] - 22s 503us/sample - loss: 0.0346 - acc: 0.9857
Epoch 4/5
43625/43625==============================] - 22s 499us/sample - loss: 0.0341 - acc: 0.9857
Epoch 5/5
43625/43625==============================] - 22s 503us/sample - loss: 0.0341 - acc: 0.9857


In [ ]:
words_syntactic_model_2.fit([text_12, text_22, syntactic_12, syntactic_22, relation_2], label_gilang, epochs=10)
words_syntactic_model_2.save('models/coreference_classifiers/words_syntactic_gilang_20.model')

Epoch 1/10
43625/43625==============================] - 22s 500us/sample - loss: 0.0327 - acc: 0.9857
Epoch 2/10
43625/43625==============================] - 22s 502us/sample - loss: 0.0324 - acc: 0.9857
Epoch 3/10
43625/43625==============================] - 22s 504us/sample - loss: 0.0323 - acc: 0.9857
Epoch 4/10
43625/43625==============================] - 22s 502us/sample - loss: 0.0325 - acc: 0.9857
Epoch 5/10
43625/43625==============================] - 22s 502us/sample - loss: 0.0328 - acc: 0.9857
Epoch 6/10
43625/43625==============================] - 22s 505us/sample - loss: 0.0318 - acc: 0.9857
Epoch 7/10
43625/43625==============================] - 22s 503us/sample - loss: 0.0306 - acc: 0.9857
Epoch 8/10
43625/43625==============================] - 22s 499us/sample - loss: 0.0299 - acc: 0.9857
Epoch 9/10
43625/43625==============================] - 22s 503us/sample - loss: 0.0314 - acc: 0.9857
Epoch 10/10
43625/43625==============================] - 22s 501us/sample - loss: 

### Context + Syntactic

In [ ]:
context_syntactic_model_2 = context_syntactic_model_builder.create_model()

In [ ]:
context_syntactic_model_2.fit([prev_12, prev_22, next_12, next_22, syntactic_12, syntactic_22, relation_2], label_gilang, epochs=5)
context_syntactic_model_2.save('models/coreference_classifiers/context_syntactic_gilang_5.model')

43625/43625==============================] - 29s 675us/sample - loss: 0.0472 - acc: 0.9857
Epoch 4/5
43625/43625==============================] - 29s 675us/sample - loss: 0.0435 - acc: 0.9857
Epoch 5/5
43625/43625==============================] - 29s 673us/sample - loss: 0.0414 - acc: 0.9857


In [ ]:
context_syntactic_model_2.fit([prev_12, prev_22, next_12, next_22, syntactic_12, syntactic_22, relation_2], label_gilang, epochs=5)
context_syntactic_model_2.save('models/coreference_classifiers/context_syntactic_gilang_10.model')

Epoch 1/5
43625/43625==============================] - 29s 671us/sample - loss: 0.0388 - acc: 0.9857
Epoch 2/5
43625/43625==============================] - 29s 675us/sample - loss: 0.0383 - acc: 0.9857
Epoch 3/5
43625/43625==============================] - 29s 674us/sample - loss: 0.0370 - acc: 0.9857
Epoch 4/5
43625/43625==============================] - 29s 668us/sample - loss: 0.0359 - acc: 0.9857
Epoch 5/5
43625/43625==============================] - 30s 676us/sample - loss: 0.0357 - acc: 0.9857


In [ ]:
context_syntactic_model_2.fit([prev_12, prev_22, next_12, next_22, syntactic_12, syntactic_22, relation_2], label_gilang, epochs=10)
context_syntactic_model_2.save('models/coreference_classifiers/context_syntactic_gilang_20.model')

Epoch 1/10
43625/43625==============================] - 29s 673us/sample - loss: 0.0347 - acc: 0.9858
Epoch 2/10
43625/43625==============================] - 29s 676us/sample - loss: 0.0343 - acc: 0.9857
Epoch 3/10
43625/43625==============================] - 30s 676us/sample - loss: 0.0334 - acc: 0.9857
Epoch 4/10
43625/43625==============================] - 29s 675us/sample - loss: 0.0320 - acc: 0.9857
Epoch 5/10
43625/43625==============================] - 29s 676us/sample - loss: 0.0325 - acc: 0.9855
Epoch 6/10
43625/43625==============================] - 29s 675us/sample - loss: 0.0313 - acc: 0.9857
Epoch 7/10
43625/43625==============================] - 29s 676us/sample - loss: 0.0328 - acc: 0.9859
Epoch 8/10
43625/43625==============================] - 29s 674us/sample - loss: 0.0298 - acc: 0.9856
Epoch 9/10
43625/43625==============================] - 29s 674us/sample - loss: 0.0302 - acc: 0.9856
Epoch 10/10
43625/43625==============================] - 29s 674us/sample - loss: 

### Words + Context + Syntactic

In [ ]:
words_context_syntactic_model_2 = words_context_syntactic_model_builder.create_model()

In [ ]:
words_context_syntactic_model_2.fit([text_12, text_22, prev_12, prev_22, next_12, next_22, syntactic_12, syntactic_22, relation_2], label_gilang, epochs=5)
words_context_syntactic_model_2.save('models/coreference_classifiers/words_context_syntactic_gilang_5.model')

Epoch 1/5
43625/43625==============================] - 39s 897us/sample - loss: 0.0612 - acc: 0.9845
Epoch 2/5
43625/43625==============================] - 37s 853us/sample - loss: 0.0449 - acc: 0.9857
Epoch 3/5
43625/43625==============================] - 37s 852us/sample - loss: 0.0408 - acc: 0.9857
Epoch 4/5
43625/43625==============================] - 37s 850us/sample - loss: 0.0378 - acc: 0.9857
Epoch 5/5
43625/43625==============================] - 37s 849us/sample - loss: 0.0375 - acc: 0.9857


In [ ]:
words_context_syntactic_model_2.fit([text_12, text_22, prev_12, prev_22, next_12, next_22, syntactic_12, syntactic_22, relation_2], label_gilang, epochs=5)
words_context_syntactic_model_2.save('models/coreference_classifiers/words_context_syntactic_gilang_10.model')

Epoch 1/5
43625/43625==============================] - 37s 851us/sample - loss: 0.0354 - acc: 0.9857
Epoch 2/5
43625/43625==============================] - 37s 853us/sample - loss: 0.0340 - acc: 0.9857
Epoch 3/5
43625/43625==============================] - 37s 853us/sample - loss: 0.0328 - acc: 0.9857
Epoch 4/5
43625/43625==============================] - 37s 850us/sample - loss: 0.0316 - acc: 0.9857
Epoch 5/5
43625/43625==============================] - 37s 851us/sample - loss: 0.0312 - acc: 0.9857


In [ ]:
words_context_syntactic_model_2.fit([text_12, text_22, prev_12, prev_22, next_12, next_22, syntactic_12, syntactic_22, relation_2], label_gilang, epochs=10)
words_context_syntactic_model_2.save('models/coreference_classifiers/words_context_syntactic_gilang_20.model')

Epoch 1/10
43625/43625==============================] - 37s 846us/sample - loss: 0.0315 - acc: 0.9857
Epoch 2/10
43625/43625==============================] - 37s 852us/sample - loss: 0.0305 - acc: 0.9857
Epoch 3/10
43625/43625==============================] - 37s 850us/sample - loss: 0.0298 - acc: 0.9857
Epoch 4/10
43625/43625==============================] - 37s 850us/sample - loss: 0.0289 - acc: 0.9857
Epoch 5/10
43625/43625==============================] - 37s 848us/sample - loss: 0.0292 - acc: 0.9858
Epoch 6/10
43625/43625==============================] - 37s 847us/sample - loss: 0.0281 - acc: 0.9856
Epoch 7/10
42976/43625============================>.] - ETA: 0s - loss: 0.0279 - acc: 0.9858